<a href="https://colab.research.google.com/github/luisdiaz1997/DataScienceCourse/blob/master/Week_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torchvision
from tqdm.autonotebook import tqdm

In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)

In [0]:
!nvidia-smi

In [0]:
! pip install -q kaggle
! mkdir ~/.kaggle

In [0]:
! cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download -d chetankv/dogs-cats-images
!unzip dogs-cats-images.zip

In [0]:
import torchvision.transforms as transforms
data_transforms = transforms.Compose([
                    transforms.Resize([64,64]),
                    transforms.ToTensor()])

In [0]:
train_dataset = torchvision.datasets.ImageFolder('dataset/training_set', transform=data_transforms)
train_dataloader = DataLoader(train_dataset, batch_size=32,
                        shuffle=True, num_workers=4)


test_dataset = torchvision.datasets.ImageFolder('dataset/test_set', transform=data_transforms)
test_dataloader = DataLoader(test_dataset, batch_size=4,
                        shuffle=False, num_workers=4)

In [0]:
data_iter = iter(train_dataloader)
images, labels = data_iter.next()

In [0]:
def imshow(img, one_channel = True):
    npimg = img.cpu().numpy()
    if one_channel:
      npimg = np.squeeze(npimg)
      plt.imshow(npimg, cmap = 'gray')
    else:
      plt.imshow(np.transpose(npimg, axes = (1, 2, 0)))

In [0]:
imgrid = torchvision.utils.make_grid(images)

In [0]:
imshow(imgrid, one_channel= False)

In [0]:
class Neural(torch.nn.Module):
    def __init__(self):
        super(Neural, self).__init__()

        
        
        
    def forward(self, inputs):
        
        
        return x

In [0]:
from torchsummary import summary
summary(model, input_size=(3, 64, 64))

In [0]:
def images_to_probs(model, images):
    '''
    Generates predictions and corresponding probabilities from a trained
    network and a list of images
    '''
    output = model(images)
    output = torch.softmax(output, 1)

    probs, preds = torch.max(output, 1)

    
    
    return probs, preds

In [0]:
def plot_classes_preds(model, images, labels):
    probs, preds = images_to_probs( model, images)
    # plot the images in the batch, along with predicted and true labels
    fig = plt.figure(figsize=(10, 4))
    for idx in range(4):
        ax = fig.add_subplot(1, 4, idx+1, xticks=[], yticks=[])
        if images.shape[1] == 3:
          imshow(images[idx],one_channel= False)
        else:
          imshow(images[idx])
        ax.set_title("{0}, {1:.1f}%\n(label: {2})".format(
            preds[idx],
            probs[idx]*100,
            labels[idx]),
                    color=("green" if preds[idx]==labels[idx].item() else "red"))
    return fig

In [0]:
fig = plot_classes_preds(model, images.to(device), labels.to(device))

In [0]:
%load_ext tensorboard

In [0]:
from torch.utils.tensorboard import SummaryWriter

# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter('runs/cat_dog_cnn')

In [0]:
writer.add_image('cat_dog', imgrid)
writer.add_graph(model, images.to(device))
writer.close()

In [0]:
%tensorboard --logdir=runs

In [0]:
%tensorboard --logdir=runs

In [0]:
def fit(model, dataloader, loss, optimizer, device, writer =None, epochs= 5, print_every = 100):

    model.train()
    for epoch in tqdm(range(epochs)):
      running_cost = 0.0
      for i, data in enumerate(tqdm(dataloader, leave = False)):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        J = loss(outputs, labels)
        J.backward()
        optimizer.step()
        if (writer != None):
          running_cost += J.item()
          if i % print_every == (print_every-1):
            writer.add_scalar('training_cost',
                              running_cost / print_every,
                              epoch * len(dataloader) + i)
            
            writer.add_figure('predictions vs. actuals',
                              plot_classes_preds(model, inputs, labels),
                              global_step=epoch * len(dataloader) + i)
            running_cost = 0.0

In [0]:
def accuracy(model, dataloader, device):
  model.eval()
  correct = 0
  total = 0
  with torch.no_grad():
      for data in dataloader:
          images, labels = data[0].to(device), data[1].to(device)
          outputs = model(images)
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()

  print('Accuracy of the network on the images: %d %%' % (
      100 * correct / total))

In [0]:
accuracy(model, train_dataloader, device)

In [0]:
accuracy(model, test_dataloader, device)

In [0]:
visualisation = {}

def hook_fn(m, i, o):
  visualisation[m] = o 

def get_all_layers(net):
  for name, layer in net._modules.items():
    layer.register_forward_hook(hook_fn)
    
get_all_layers(model)


In [0]:
model(images.to(device))

In [0]:
first_out = visualisation[list(visualisation.keys())[6]]

In [0]:
first_out[0].shape

In [0]:
k = 1
plt.figure(1, figsize = (2, 2) )
plt.imshow(images[k].detach().numpy().transpose(1, 2, 0))
plt.figure(2, figsize = (10, 10))
for i in range(16):
  plt.subplot(4, 4, i+1)
  plt.imshow(first_out[k][i].cpu().detach().numpy())
  plt.axis('off')